# 1. Importing Libraries:


In [1]:
# Import necessary libraries
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, MobileNet
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.utils.class_weight import compute_class_weight
import numpy as np

# 2. Data Augmentation and Splitting:


In [3]:
# Data augmentation and normalization for training and validation
datagen = ImageDataGenerator(
    rescale=1./255,            # Normalize images
    rotation_range=20,         # Random rotations
    width_shift_range=0.2,     # Horizontal shift
    height_shift_range=0.2,    # Vertical shift
    zoom_range=0.2,            # Random zoom
    horizontal_flip=True,      # Flip horizontally
    validation_split=0.2       # Split 20% for validation
)

# Load training data
train_data = datagen.flow_from_directory(
    r'D:\Ammar\AMIT Diploma\Machine Leaning\ML Supervised (challenge)\archive\Wonders of World\Wonders of World',          # Path to your images
    target_size=(224, 224),    # Resize images to 224x224
    batch_size=32,             # Batch size
    class_mode='categorical',  # Categorical labels
    subset='training'          # Training data
)

# Load validation data
val_data = datagen.flow_from_directory(
    r'D:\Ammar\AMIT Diploma\Machine Leaning\ML Supervised (challenge)\archive\Wonders of World\Wonders of World',          # Path to your images
    target_size=(224, 224),    # Resize images to 224x224
    batch_size=32,             # Batch size
    class_mode='categorical',  # Categorical labels
    subset='validation'        # Validation data
)


Found 3082 images belonging to 12 classes.
Found 764 images belonging to 12 classes.


# 3. Class Weight Calculation:


In [4]:
# Compute class weights to handle imbalance
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_data.classes),
    y=train_data.classes
)
class_weights = dict(enumerate(class_weights))  # Convert to dictionary

# 4. Early Stopping Callback:


In [5]:
# Early stopping to prevent overfitting
early_stopping = EarlyStopping(
    monitor='val_loss',    # Monitor validation loss
    patience=3,            # Stop after 3 epochs with no improvement
    restore_best_weights=True  # Restore the best weights
)

# 5. Model 1: VGG16 Transfer Learning:


In [6]:
# Load pre-trained VGG16 without top layers
vgg16_base = VGG16(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
for layer in vgg16_base.layers:
    layer.trainable = False

# Add custom classification layers
x = Flatten()(vgg16_base.output)
x = Dense(128, activation='relu')(x)
output = Dense(train_data.num_classes, activation='softmax')(x)

# Define the VGG16 model
vgg16_model = Model(inputs=vgg16_base.input, outputs=output)

# Compile the model
vgg16_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
vgg16_history = vgg16_model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    class_weight=class_weights,
    callbacks=[early_stopping]
)


Epoch 1/10


C:\Users\aliem\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


97/97 ━━━━━━━━━━━━━━━━━━━━ 304s 3s/step - accuracy: 0.4869 - loss: 1.8360 - val_accuracy: 0.7723 - val_loss: 0.6727
Epoch 2/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 306s 3s/step - accuracy: 0.7907 - loss: 0.6706 - val_accuracy: 0.8364 - val_loss: 0.5075
Epoch 3/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 307s 3s/step - accuracy: 0.8397 - loss: 0.4778 - val_accuracy: 0.8717 - val_loss: 0.3939
Epoch 4/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 308s 3s/step - accuracy: 0.8558 - loss: 0.4425 - val_accuracy: 0.8429 - val_loss: 0.4843
Epoch 5/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 309s 3s/step - accuracy: 0.8691 - loss: 0.3981 - val_accuracy: 0.8312 - val_loss: 0.6398
Epoch 6/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 310s 3s/step - accuracy: 0.8734 - loss: 0.3619 - val_accuracy: 0.8442 - val_loss: 0.5577


# 6. Model 2: ResNet50 Transfer Learning:


In [9]:
# Load pre-trained ResNet50 without top layers
resnet_base = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
for layer in resnet_base.layers:
    layer.trainable = False

# Add custom classification layers
x = Flatten()(resnet_base.output)
x = Dense(128, activation='relu')(x)
output = Dense(train_data.num_classes, activation='softmax')(x)

# Define the ResNet50 model
resnet_model = Model(inputs=resnet_base.input, outputs=output)

# Compile the model
resnet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
resnet_history = resnet_model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    class_weight=class_weights,
    callbacks=[early_stopping]
)


94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 31s 0us/step
Epoch 1/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 132s 1s/step - accuracy: 0.0932 - loss: 5.4074 - val_accuracy: 0.1152 - val_loss: 2.4705
Epoch 2/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 127s 1s/step - accuracy: 0.0931 - loss: 2.4584 - val_accuracy: 0.1021 - val_loss: 2.4839
Epoch 3/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 129s 1s/step - accuracy: 0.1090 - loss: 2.4772 - val_accuracy: 0.1034 - val_loss: 2.4857


# 7. Model 3: MobileNet Transfer Learning:


In [7]:
# Load pre-trained MobileNet without top layers
mobilenet_base = MobileNet(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze base model layers
for layer in mobilenet_base.layers:
    layer.trainable = False

# Add custom classification layers
x = Flatten()(mobilenet_base.output)
x = Dense(128, activation='relu')(x)
output = Dense(train_data.num_classes, activation='softmax')(x)

# Define the MobileNet model
mobilenet_model = Model(inputs=mobilenet_base.input, outputs=output)

# Compile the model
mobilenet_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the model
mobilenet_history = mobilenet_model.fit(
    train_data,
    validation_data=val_data,
    epochs=10,
    class_weight=class_weights,
    callbacks=[early_stopping]
)



17225924/17225924 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step
Epoch 1/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 70s 670ms/step - accuracy: 0.4976 - loss: 7.8349 - val_accuracy: 0.8442 - val_loss: 0.5305
Epoch 2/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 66s 655ms/step - accuracy: 0.8759 - loss: 0.4252 - val_accuracy: 0.8822 - val_loss: 0.4498
Epoch 3/10
97/97 ━━━━━━━━━━━━━━━━━━━━ 66s 657ms/step - accuracy: 0.8950 - loss: 0.3398 - val_accuracy: 0.8717 - val_loss: 0.5164


# 8. Model Evaluation:


In [10]:
# Evaluate VGG16 model
vgg16_scores = vgg16_model.evaluate(val_data)
print(f"VGG16 Validation Accuracy: {vgg16_scores[1] * 100:.2f}%")

# Evaluate ResNet50 model
resnet_scores = resnet_model.evaluate(val_data)
print(f"ResNet50 Validation Accuracy: {resnet_scores[1] * 100:.2f}%")

# Evaluate MobileNet model
mobilenet_scores = mobilenet_model.evaluate(val_data)
print(f"MobileNet Validation Accuracy: {mobilenet_scores[1] * 100:.2f}%")


24/24 ━━━━━━━━━━━━━━━━━━━━ 60s 2s/step - accuracy: 0.8609 - loss: 0.4213
VGG16 Validation Accuracy: 87.57%
24/24 ━━━━━━━━━━━━━━━━━━━━ 25s 1s/step - accuracy: 0.1141 - loss: 2.4700
ResNet50 Validation Accuracy: 12.30%
24/24 ━━━━━━━━━━━━━━━━━━━━ 13s 520ms/step - accuracy: 0.8548 - loss: 0.5372
MobileNet Validation Accuracy: 85.47%


# 9. Model Comparison:


In [11]:
# Compare models based on validation accuracy
models = {
    'VGG16': vgg16_scores[1] * 100,
    'ResNet50': resnet_scores[1] * 100,
    'MobileNet': mobilenet_scores[1] * 100
}

best_model = max(models, key=models.get)

print(f"The best model is {best_model} with an accuracy of {models[best_model]:.2f}%")


The best model is VGG16 with an accuracy of 87.57%


In [16]:
# Create ImageDataGenerator for test data (without augmentation)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load test data
test_data = test_datagen.flow_from_directory(
    r'D:\Ammar\AMIT Diploma\Machine Leaning\ML Supervised (challenge)\archive\Wonders of World\Wonders of World',  # Replace with the actual path
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)


Found 3846 images belonging to 12 classes.


In [17]:
vgg16_scores = vgg16_model.evaluate(test_data)
resnet_scores = resnet_model.evaluate(test_data)
mobilenet_scores = mobilenet_model.evaluate(test_data)

C:\Users\aliem\AppData\Roaming\Python\Python311\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


121/121 ━━━━━━━━━━━━━━━━━━━━ 283s 2s/step - accuracy: 0.9281 - loss: 0.2387
121/121 ━━━━━━━━━━━━━━━━━━━━ 117s 966ms/step - accuracy: 0.1048 - loss: 2.4686
121/121 ━━━━━━━━━━━━━━━━━━━━ 40s 332ms/step - accuracy: 0.8788 - loss: 0.3946


In [40]:
print(f"The best model is `vgg16` model with an accuracy of {vgg16_scores[1]:.2f}%")

The best model is `vgg16` model with an accuracy of 0.94%


In [19]:
vgg16_model.save('Wonders_of_the_world_classfication.h5')

- The best model is `vgg16_model`:
- with accuarcy **93%** on the test data 
- and **87.6%** accuarcy on the validation data

### Project Summary: Classification of Wonders of the World Using Transfer Learning

#### Objective:
The project aimed to classify images of various wonders of the world using deep learning techniques, specifically leveraging transfer learning with pre-trained models.

#### Methodology:

1. **Importing Libraries**:
   - Utilized TensorFlow and Keras for building and training the models, along with libraries for data preprocessing and handling class imbalance.

2. **Data Augmentation and Splitting**:
   - Employed `ImageDataGenerator` for data augmentation to enhance the training dataset by applying transformations such as rotation, shifting, zooming, and flipping.
   - Split the dataset into training (80%) and validation (20%) sets, ensuring a balanced representation of classes.

3. **Class Weight Calculation**:
   - Computed class weights to address any class imbalance in the dataset, ensuring that the model pays equal attention to all classes during training.

4. **Early Stopping Callback**:
   - Implemented early stopping to prevent overfitting by monitoring validation loss and stopping training when no improvement was observed for three consecutive epochs.

5. **Model Development**:
   - **Model 1: VGG16**:
     - Loaded the pre-trained VGG16 model, froze its layers, and added custom classification layers. Achieved a validation accuracy of 87.57%.
   - **Model 2: ResNet50**:
     - Loaded the ResNet50 model, froze its layers, and added custom layers. However, it performed poorly with a validation accuracy of only 12.30%.
   - **Model 3: MobileNet**:
     - Loaded the MobileNet model, froze its layers, and added custom layers, achieving a validation accuracy of 85.47%.

6. **Model Evaluation**:
   - Evaluated all three models on the validation dataset, with VGG16 outperforming the others significantly.
   - The best model, VGG16, was further evaluated on a separate test dataset, achieving an impressive accuracy of 93%.

7. **Model Saving**:
   - The VGG16 model was saved for future use, with a recommendation to use the native Keras format for model saving.

#### Results:
- The VGG16 model emerged as the best performer with a validation accuracy of 87.57% and a test accuracy of 93%. The ResNet50 model underperformed significantly, while MobileNet showed moderate performance.

#### Conclusion:
The project successfully demonstrated the effectiveness of transfer learning in image classification tasks, particularly with the VGG16 model. The results indicate that using pre-trained models can significantly enhance classification accuracy, especially in scenarios with limited training data. Future work could explore fine-tuning the models and experimenting with additional architectures to further improve performance.